In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.vgg16 import VGG16
import cv2

C:\Users\felix\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Constants

In [2]:
IMAGE_SIZE = 150
LEARNING_RATE = 1e-3
TRAIN_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\train_data.npy'
TEST_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\test_data.npy'
IMAGE_TEST = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\nn\\test img\\img.JPG'
ONE_HOT = np.array(['American Shorthair', 'Angora', 'Ashera', 'British Shorthair',
                    'Exotic', 'Himalayan', 'Maine Coon', 'Persian', 'Ragdoll', 'Siamese', 'Sphynx'])

# Creating model

In [3]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

model = Sequential()

for layer in vgg.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False


model.add(Flatten())

model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.8))

model.add(Dense(11, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       295168    
__________

# Load and Process Data

In [4]:
train_data, test_data = np.load(TRAIN_PATH), np.load(TEST_PATH)

train_features = train_data[:, 0]
train_features = np.array([list(i) for i in train_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
train_labels = train_data[:, 1]
train_labels = np.array([list(i) for i in train_labels])


test_features = test_data[:, 0]
test_features = np.array([list(i) for i in test_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
test_labels = test_data[:, 1]
test_labels = np.array([list(i) for i in test_labels])

# Training Model

In [5]:
tb = TensorBoard()
tb.set_model('VGG16(high_drop)')

In [18]:
model.fit(x=train_features,
          y=train_labels,
          batch_size=64,
          epochs=300,
          validation_data=(test_features, test_labels),
          verbose=1,
          callbacks=[tb],
          initial_epoch=140
         )

Train on 4514 samples, validate on 674 samples
Epoch 141/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1445 - acc: 0.6176 - val_loss: 1.6772 - val_acc: 0.5593
Epoch 142/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1675 - acc: 0.5981 - val_loss: 1.6802 - val_acc: 0.5608
Epoch 143/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1424 - acc: 0.6072 - val_loss: 1.7322 - val_acc: 0.5445
Epoch 144/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1952 - acc: 0.5961 - val_loss: 1.6584 - val_acc: 0.5282
Epoch 145/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.1928 - acc: 0.5953 - val_loss: 1.6994 - val_acc: 0.5490
Epoch 146/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1289 - acc: 0.6163 - val_loss: 1.6559 - val_acc: 0.5579
Epoch 147/300
4514/4514 [==============================] - 27s 6ms/step - loss: 1.1570 - acc: 0.6066 - val_loss: 1.7449 - val

4514/4514 [==============================] - 28s 6ms/step - loss: 1.0222 - acc: 0.6486 - val_loss: 1.7649 - val_acc: 0.5549
Epoch 201/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0738 - acc: 0.6429 - val_loss: 1.7380 - val_acc: 0.5697
Epoch 202/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.1135 - acc: 0.6265 - val_loss: 1.8715 - val_acc: 0.5401
Epoch 203/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0747 - acc: 0.6362 - val_loss: 1.8234 - val_acc: 0.5608
Epoch 204/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0309 - acc: 0.6526 - val_loss: 1.8180 - val_acc: 0.5504
Epoch 205/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0276 - acc: 0.6544 - val_loss: 1.9701 - val_acc: 0.5415
Epoch 206/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0485 - acc: 0.6498 - val_loss: 1.8681 - val_acc: 0.5564
Epoch 207/300
4514/4514 [=======================

4514/4514 [==============================] - 28s 6ms/step - loss: 0.9895 - acc: 0.6675 - val_loss: 1.9283 - val_acc: 0.5534
Epoch 260/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0238 - acc: 0.6622 - val_loss: 1.9626 - val_acc: 0.5401
Epoch 261/300
4514/4514 [==============================] - 28s 6ms/step - loss: 1.0007 - acc: 0.6648 - val_loss: 2.0519 - val_acc: 0.5326
Epoch 262/300
4514/4514 [==============================] - 28s 6ms/step - loss: 0.9588 - acc: 0.6741 - val_loss: 2.0729 - val_acc: 0.5608
Epoch 263/300
4514/4514 [==============================] - 28s 6ms/step - loss: 0.9816 - acc: 0.6704 - val_loss: 2.0525 - val_acc: 0.5490
Epoch 264/300
4514/4514 [==============================] - 28s 6ms/step - loss: 0.9562 - acc: 0.6755 - val_loss: 2.0040 - val_acc: 0.5371
Epoch 265/300
4514/4514 [==============================] - 28s 6ms/step - loss: 0.9244 - acc: 0.6887 - val_loss: 2.0072 - val_acc: 0.5519
Epoch 266/300
4514/4514 [=======================

In [19]:
model.save('.\\models\\VGG16(high_drop)\\VGG16(high_drop)-300_epochs.h5')

# Testing

In [20]:
def resize_square(image_current_path, shape):
    """Resize the image to a square shape keeping it's original proportion
    and filling the rest of image with black pads to keep it square.

    :param image_current_path: the local path where the image is.
    :type image_current_path: OS string
    :param shape: number of pixels per axis of the converted image
    :type shape: int
    :param save_path: path where the converted image must be saved
    :type save_path: OS string
    """

    original_image = cv2.imread(image_current_path, 1)
    original_size = original_image.shape[:2]  # (height, width)

    ratio = float(shape)/max(original_size)

    format_size = tuple(reversed([int(i*ratio)
                                  for i in original_size]))  # (width, height)

    resized_image = cv2.resize(original_image, format_size)

    d_w = shape - format_size[0]
    d_h = shape - format_size[1]

    top, bottom = d_h//2, d_h-(d_h//2)
    left, right = d_w//2, d_w-(d_w//2)

    color = [0, 0, 0]

    squared_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return squared_image

In [21]:
img = resize_square(IMAGE_TEST, IMAGE_SIZE)
img = np.array(img)/255

out = model.predict([img.reshape(-1, 150, 150, 3)])[0]

for n in range(11):
    print('{} - {:2.2f}%'.format(ONE_HOT[n], out[n]*100))

American Shorthair - 0.00%
Angora - 62.28%
Ashera - 0.00%
British Shorthair - 0.00%
Exotic - 0.01%
Himalayan - 4.32%
Maine Coon - 0.03%
Persian - 2.12%
Ragdoll - 31.17%
Siamese - 0.06%
Sphynx - 0.00%
